In [1]:
from GCHPc.gchpp_simple_model import GCHPpType, GCHPp, calculate_ht_mass_flow, calculate_lt_supply, calculate_orc_mass_flow, calculate_t_bo
from GCHPc.demand import ComodityType, Demand
from GCHPc.brine_properties import Brine
from GCHPc.orc_m01 import BinaryType, ORC
import numpy as np

In [4]:
gchpp = GCHPp(name='test', m_bi = 47.1, t_bi = 157.5, t_bo = 35., type=GCHPpType.parallel_serries)
gchpp.brine = Brine(name='balmatt', tds=.031, temperature=gchpp.t_bi, pressure=4.86)
gchpp.ht_demand = Demand(comodity_type=ComodityType.HEAT, curve=np.array([3]), t_supply=120, t_return=80)
gchpp.lt_demand = Demand(comodity_type=ComodityType.HEAT, curve=np.array([5]), t_supply=60, t_return=40)
gchpp.orc = ORC(name='orc', type=BinaryType.ocr_acc, t_bi=gchpp.t_bi)
print(gchpp.orc.__dict__)

{'name': 'orc', 'type': <BinaryType.ocr_acc: 0>, 't_bi': 157.5, 't_bo': 65.9, 'n': 12.77}


In [5]:
fulload = 8760 * .8

out = calculate_ht_mass_flow(gchpp=gchpp,cutoff=.6)
gchpp.m_ht = out['ht_flow']
gchpp.ht_supply = out['ht_supply']

out = calculate_lt_supply(gchpp=gchpp, cuttoff=1.)
gchpp.lt_supply = out['lt_supply']

out = calculate_orc_mass_flow(gchpp=gchpp)
gchpp.m_orc = out

Eheat = np.average(gchpp.ht_supply) + np.average(gchpp.lt_supply)
Eheat *= fulload
Eelec = np.average(gchpp.orc.calculate_output(flow_rate=out, brine=gchpp.brine)) * fulload

print('heat => {:,.0f}'.format(Eheat) + ' MWh')
print('heat => {:,.0f}'.format(Eelec) + ' MWh')
print ('T out => {:.1f}'.format(np.average(calculate_t_bo(gchpp))) + '°C')

heat => 39,238 MWh
heat => 13,730 MWh
T out => 47.6°C
